In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse sentencepiece sacremoses modelscope datasets peft gradio pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 890.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import json
import re
import sqlite3
import pandas as pd
import sqlparse
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import gradio as gr

In [ ]:
# Define paths
data_path = "/content/sample_data/AR_spider.jsonl"
output_dir = "/content/sql_generation_output"
os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist

# Load the Arabic Spider dataset
def load_arabic_spider_data(file_path):
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                item = json.loads(line)
                # Validate required fields exist
                if all(key in item for key in ['question', 'arabic', 'db_id', 'query']):
                    data.append(item)
            except json.JSONDecodeError as e:
                print(f"Skipping invalid JSON line: {e}")
    return data

# Load and verify data
spider_data = load_arabic_spider_data(data_path)
print(f"Successfully loaded {len(spider_data)} Arabic Spider examples")
print("\nSample entry:")
print(json.dumps(spider_data[0], indent=2, ensure_ascii=False))

Successfully loaded 6396 Arabic Spider examples

Sample entry:
{
  "question": "How many heads of the departments are older than 56 ?",
  "query": "SELECT count(*) FROM head WHERE age  >  56",
  "arabic": "كم عدد رؤساء الأقسام الذين تزيد أعمارهم عن 56 سنة؟",
  "db_id": "department_management"
}


In [ ]:
model_name = "motherduckdb/DuckDB-NSQL-7B-v0.1"  # Specialized for SQL generation

# Configure 4-bit quantization for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load tokenizer with Arabic support
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="left"  # Important for generation
)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

# Load model with QLoRA configuration
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Prepare model for 4-bit training
model = prepare_model_for_kbit_training(model)

# Configure LoRA for efficient fine-tuning
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

print("Model and tokenizer successfully loaded!")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Model and tokenizer successfully loaded!
Trainable parameters: 8388608


In [ ]:
from datasets import Dataset
import pandas as pd

def create_arabic_sql_prompt(item):
    """Clean prompt format without extra padding"""
    return f"""### السؤال:
{item['arabic']}

### قاعدة البيانات:
{item['db_id']}

### SQL:
{item['query']}"""

# 1. Create clean datasets
train_data = [{"text": create_arabic_sql_prompt(item)} for item in spider_data[:5000]]
val_data = [{"text": create_arabic_sql_prompt(item)} for item in spider_data[5000:]]

# 2. Proper tokenization with smart padding
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors="pt",
        add_special_tokens=True,
        pad_to_max_length=True  # Explicit padding control
    )

# 3. Create datasets with clean formatting
train_dataset = Dataset.from_pandas(pd.DataFrame(train_data)).map(
    tokenize_function,
    batched=True,
    batch_size=4,  # Smaller batches for stability
    remove_columns=["text"]  # Remove raw text to save memory
)

val_dataset = Dataset.from_pandas(pd.DataFrame(val_data)).map(
    tokenize_function,
    batched=True,
    batch_size=4,
    remove_columns=["text"]
)

# 4. Verification with clean output
sample_output = tokenizer.decode(train_dataset[0]['input_ids'], skip_special_tokens=True)
print(f"✅ Clean data prepared | Train: {len(train_dataset)} | Val: {len(val_dataset)}")
print("\nSample without padding:")
print(sample_output[:200] + "..." if len(sample_output) > 200 else sample_output)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1396 [00:00<?, ? examples/s]

✅ Clean data prepared | Train: 5000 | Val: 1396

Sample without padding:
### السؤال:
كم عدد رؤساء الأقسام الذين تزيد أعمارهم عن 56 سنة؟

### قاعدة البيانات:
department_management

### SQL:
SELECT count(*) FROM head WHERE age  >  56


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

# 1. Configure training with padding control
training_args = TrainingArguments(
    output_dir="./arabic_sql_results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=3e-5,
    fp16=True,
    save_steps=500,
    logging_steps=25,
    optim="adamw_torch_fused",
    report_to="none",
    max_steps=500,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
)

# 2. Data collator with smart padding
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8  # Optimize GPU memory
)

# 3. Trainer with padding control
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

# 4. Critical model adjustments
model.config.pad_token_id = tokenizer.pad_token_id  # Explicit pad token
model.config.use_cache = False
model.enable_input_require_grads()

print("🔥 Clean Training Setup Ready 🔥")
print("Key Config:")
print(f"- Batch Size: {2} (x{4})")
print(f"- Sequence Length: 256")
print(f"- Pad Token: {tokenizer.pad_token}")

🔥 Clean Training Setup Ready 🔥
Key Config:
- Batch Size: 2 (x4)
- Sequence Length: 256
- Pad Token: </s>


In [ ]:
trainer.train()  # Start fine-tuning

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,3.107200
50,2.705600
75,1.931800
100,1.461800
125,1.265900
150,1.126300
175,1.024400
200,0.975600
225,0.903800
250,0.900500


TrainOutput(global_step=500, training_loss=1.1818122901916503, metrics={'train_runtime': 3924.7285, 'train_samples_per_second': 1.019, 'train_steps_per_second': 0.127, 'total_flos': 4.064746143744e+16, 'train_loss': 1.1818122901916503, 'epoch': 0.8})

In [ ]:
# Save the model and tokenizer
model.save_pretrained("./arabic_sql_model")
tokenizer.save_pretrained("./arabic_sql_model")

('./arabic_sql_model/tokenizer_config.json',
 './arabic_sql_model/special_tokens_map.json',
 './arabic_sql_model/tokenizer.model',
 './arabic_sql_model/added_tokens.json',
 './arabic_sql_model/tokenizer.json')

In [ ]:
def generate_sql(question, db_id):
    prompt = f"""### السؤال:
{question}

### قاعدة البيانات:
{db_id}

### SQL:"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
new_question = "كم عدد الموظفين في قسم المبيعات؟"
db_id = "department_management"
print(generate_sql(new_question, db_id))

### السؤال:
كم عدد الموظفين في قسم المبيعات؟

### قاعدة البيانات:
department_management

### SQL:
SELECT count(*) FROM employees AS T1 JOIN departments AS T2 ON T1.department_id  =  T2.department_id WHERE T2.department_name  =  "Sales";

### SQL:
SELECT count(*) FROM employees AS T1 JOIN departments AS T2 ON T1.department_id  =  T2.department_id WHERE T2.department_name  =  "IT";

### SQL:
SELECT count(*) FROM employees AS T1 JOIN departments AS T2 ON T1.department_id  =  T2.department_id WHERE T2.department_name  =  "HR";

### SQL:
SELECT count(*) FROM employees AS T1 JOIN departments AS T2 ON T1.department_id  =  T2.department_id WHERE T2.department_


In [ ]:
import gradio as gr

def gradio_inference(question, db_id):
    sql_query = generate_sql(question, db_id)
    return f"""```sql
{sql_query.split('SQL:')[1].strip()}
```"""

demo = gr.Interface(
    fn=gradio_inference,
    inputs=[
        gr.Textbox(label="السؤال بالعربية", lines=3),

    ],
    outputs=gr.Code(label="استعلام SQL", language="sql"),
    title="مولد استعلامات SQL من العربية"
)

demo.launch(share=True)  # Generates public URL

1.   ما هو المتوسط ​​لعدد الموظفين في الأقسام الذين يحتلون المرتبة بين 10 و 15؟
2.   ما هي الأسماء الرسمية للمدن التي لم تستضف أي مسابقة زراعية؟
3. ما هي حالة المدينة التي استضافت أكبر عدد من المسابقات؟
4. كم عدد الطلاب الذين يحضرون دورات اللغة الإنجليزية؟
5. ما هي تفاصيل الطالب الذي سجل لأكبر عدد من الدورات؟
6. ما هي الأجور الأعلى في قسم المحاسبة؟
7. ما هو متوسط الراتب في قسم المحاسبة؟





_________________________________________________________________________________________________